In [1]:
import pickle
import requests

In [2]:
#simple function to pickle variables for later use. save a local pickle
def save_object(obj, filename):
    '''Help: Given an object & filepath, store the object as a pickle for later use.'''
    with open(filename, 'wb') as outp:  # Overwrites any existing file.
        pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)
#and later load the file back into a variable
def load_object(filename):
    '''Help: Loads something previously pickled from the provided file path.'''
    with open(filename, 'rb') as f:
        load_test = pickle.load(f)
    return load_test

In [3]:
sets_from_file=load_object('mtg_set_codes.p')

In [4]:
len(sets_from_file)

743

In [5]:
sets_from_file[:3]

['Q08', 'SLC', 'TGN3']

In [6]:
sets3= sets_from_file[:3]

In [7]:
sets3

['Q08', 'SLC', 'TGN3']

In [8]:

card_data_list = []
SET_QUERY = 'https://api.scryfall.com/cards/search?order=set&unique=prints&include_variations=true&q=s%3A'
#user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
for mtg_set_code in sets_from_file:
    response = requests.get(SET_QUERY + mtg_set_code)
    if response.status_code != 200:
        print('Response status code ' + str(response.status_code))
        continue
    response_json = response.json()
    if response_json['object'] not in ['list', 'card']:
        print('Query returned ' + response_json['object'] + ' with code ' + response_json['code'])
        continue
    
    for response_card_data in response_json['data']:
        # in here add any data from response that is needed for the recognition
        # https://scryfall.com/docs/api/cards
        # for images use 'png' or 'large' (jpg) if storage space is an issue
        # if only card image is used, then 'art_crop' could be an option
        # https://scryfall.com/docs/api/images
        if 'image_uris' not in response_card_data or 'png' not in response_card_data['image_uris']:
            continue
        if 'nonfoil' in response_card_data and response_card_data['nonfoil']:
            card_data_list.append({'id': response_card_data['id'], 'image_uri': response_card_data['image_uris']['large'], 'name': response_card_data['name'].split(' //')[0]})
            # uncomment the rows below if you want to save the image files locally
            #if not os.path.exists(mtg_set_code):
                #os.makedirs(mtg_set_code)
            #image_size = 'large'
            #img_size=(224,312)
            # file_path = mtg_set_code +'/' + response_card_data['id'] + '.png'
            #file_path = mtg_set_code +'/' + response_card_data['id'] + '.jpg'
            #headers={'User-Agent':user_agent,} 
            #if not os.path.exists(file_path)
            #image_url=response_card_data['image_uris']['large']
            #urlretrieve(image_url, file_path)
            #request=urllib.request.Request(image_url,None,headers)
            #response = urllib.request.urlopen(request)
            #install PIL package to convert the response into a PIL Image object to further save it
            #clean_img=Image.open(response)
            #distorted_img = clean_img.resize(img_size)
            #distorted_img.save(f"{mtg_set_code}/{response_card_data['id'] }.jpg")



Response status code 404


In [9]:
len(card_data_list)

38416

In [10]:
save_object(card_data_list, 'card_data_list.p')